# Decorators

Note: 'decorators' and 'function wrappers' are the same thing.

[totorial](https://www.geeksforgeeks.org/python-decorators-a-complete-guide/)

[better tutorial](https://realpython.com/primer-on-python-decorators/)


method decorators:

- @staticmethod
- @classmethod
- @property

fucntion decorators:

- @functools.wraps
- @lru_cache
- @dataclass
- @atexit.register
- @enum.unique
- @singledispatch

## Creating a functional decorator

In [37]:
from functools import wraps

def example_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("wrapper")
        return func(*args, **kwargs)
    
    return wrapper

@example_decorator
def hello_name(name: str):
    print(f"hello {name}")

hello_name("Dirk")

wrapper
hello Dirk


In [38]:
def example_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        kwargs["name"] = f"{kwargs["name"]}!"
        return func(*args, **kwargs)
    
    return wrapper

@example_decorator
def hello_name(name: str):
    print(f"hello {name}")

hello_name(name="Dirk")

hello Dirk!


In [39]:
def food_wrapper(func):
    @wraps(func)
    def wrapper(*args, **kwargs):

        print("before function")
        response = func(*args, **kwargs)
        print("after function")

        return response
    
    return wrapper

@food_wrapper
def test(name):
    print(name)

test(name="Dirk")


before function
Dirk
after function


In [40]:
def duck_wrapper(func):
    @wraps(func)
    def wrapper(*args, **kwargs):

        print("before function")
        animal = "Duck"
        response = func(animal, *args, **kwargs)
        print("after function")

        return response
    
    return wrapper

@duck_wrapper
def fav_food(animal, style):
    return f"My favorite dish is {style} {animal}"

fav_food(style="grilled")

before function
after function


'My favorite dish is grilled Duck'

A handy implementation is to use decorators for validation.

In [41]:
import datetime
CREDENTIALS = {"user": "Dirk", "last_active": datetime.datetime(1970, 9, 23, 14, 10)}



def check_last_active(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if CREDENTIALS["last_active"] < (datetime.datetime.now() - datetime.timedelta(minutes=5)):
            raise AssertionError("You were offline for too long, please check if your registration information is still correct.")
        
        return func(*args, **kwargs)
    return wrapper

def make_post(message):
    return {"posted on": datetime.datetime.now(), "message": message}

make_post("hello")

{'posted on': datetime.datetime(2025, 4, 23, 12, 1, 28, 508694),
 'message': 'hello'}

In [42]:
@check_last_active
def make_post(message):
    return {"posted on": datetime.datetime.now(), "message": message}

make_post("hello")

AssertionError: You were offline for too long, please check if your registration information is still correct.

### Adding arguments to a decorator

https://blog.miguelgrinberg.com/post/the-ultimate-guide-to-python-decorators-part-iii-decorators-with-arguments

Solution: You just put the whole decorator function into another function, lmao.

In [ ]:
import datetime
from functools import wraps
CREDENTIALS = {"user": "Dirk", "last_active": datetime.datetime(1970, 9, 23, 14, 10)}


def get_last_active(last_activity: datetime.timedelta):
    def check_last_active(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if CREDENTIALS["last_active"] < (datetime.datetime.now() - last_activity):
                raise AssertionError("You were offline for too long, please check if your registration information is still correct.")
            
            return func(*args, **kwargs)
        return wrapper
    return check_last_active


@get_last_active(last_activity=datetime.timedelta(minutes=5))
def make_post(message):
    return {"posted on": datetime.datetime.now(), "message": message}

make_post("hello")

AssertionError: You were offline for too long, please check if your registration information is still correct.

## Creating OOP decorators

https://medium.com/data-science/python-decorators-in-oop-3189c526ead6

you can just use them af is the methods are fucncions?

In [43]:
import functools




def wrapper(method):
    @functools.wraps(method)
    def _impl(self, *method_args, **method_kwargs):
        method_output = method(self, *method_args, **method_kwargs)
        return method_output + "!"
    return _impl

class Foo:
    @wrapper
    def bar(self, word):
        return word

f = Foo()
result = f.bar("kitty")
print(result)

kitty!


My own (dumb) version. But can be converted to send info to a log.

In [48]:
def print_list(method_name):
    def get_method_name(method):
        @functools.wraps(method)
        def _impl(self, *method_args, **method_kwargs):
            method_output = method(self, *method_args, **method_kwargs)
            message = f"""Action:\n
                \tCurrent class: {type(self).__name__}
                \tCurrent method: {method_name}
                \tCurrent list: {str(self.list)}\n
                \tPrinting to logfile: {self.logfile}"""
            print(message)
            return method_output
        return _impl
    return get_method_name

class MovieList:
    logfile = ""
    list = []
    def __init__(self, logfile: str):
        print("initiated list")
        MovieList.logfile = logfile
    
    @print_list("add_to_list") # I don't know how to get method name, might be impossible
    def add_to_list(self, movie_id: str):
        MovieList.list.append(movie_id)


list = MovieList("handylogfile.txt")
list.add_to_list("tt5655")
list.add_to_list("tt7657")

initiated list
Action:

                	Current class: MovieList
                	Current method: add_to_list
                	Current list: ['tt5655']

                	Printing to logfile: handylogfile.txt
Action:

                	Current class: MovieList
                	Current method: add_to_list
                	Current list: ['tt5655', 'tt7657']

                	Printing to logfile: handylogfile.txt


- @lru_cache
- @dataclass
- @atexit.register
- @enum.unique
- @singledispatch

## @singledispatch

For lack of a better explanation: This is basically overloading a function based on the input type.
So if the function get's a string, if can do something different than when it get's an integer (or classes).

In [ ]:
from functools import singledispatch
from typing import Union

@singledispatch
def shout_type(type_input: Union[str, int, float]):
    raise NotImplementedError(f"I don't know this type")

@shout_type.register
def _(type_input: str):
    print("Nice string of words you have there!")

@shout_type.register
def _(type_input: int):
    print("Nice integer my dude!")

@shout_type.register
def _(type_input: float):
    print("Damn, nice float!")

shout_type(7)
shout_type(7.1)
shout_type("hello")

Nice integer my dude!
Damn, nice float!
Nice string of words you have there!


## @dataclass

This is almost how databases are made in django. I know most of it, but will recap a few things.

Let's check if the frozen parameter is immutable.

In [21]:
from dataclasses import dataclass

@dataclass(frozen=True)
class User:
    email: str
    name: str
    age: int
    height: float

    def get_height(self):
        return self.email

    # def __init__(self, email: str, name: str, age: int, height: float):
    #     self.email = email
    #     self.name = name
    #     self.age = age
    #     self.height = height

programmer = User(
    email="Rando@hotmail.com",
    name="Rando",
    age=7,
    height=1.65)

programmer.email = 1
programmer.email

FrozenInstanceError: cannot assign to field 'email'

I already know you can override the dunders, but let's check the property decorator.

In [22]:
@dataclass
class User:

    def __init__(self, email: str, name: str):
        self._email = email
        self._name = name
    
    @property
    def email(self):
        print("getting the value of email")
        return self._email

    @email.setter
    def email(self, value):
        print("setting the value of email")
        self._email = value
    
    @email.deleter
    def email(self):
        print("deleting the value of email")
        del self._email

programmer = User(email="Rando@hotmail.com",name="Rando")

programmer.email
print("1: -----------------")
programmer.email = "a"
print("2: -----------------")
del programmer.email
print("3: -----------------")


getting the value of email
1: -----------------
setting the value of email
2: -----------------
deleting the value of email
3: -----------------


Work's as expected, this is awesome.